In [2]:
import random
class BlackJackEnv:

  rCount = 0
  roundStarted = False

  def __str__(self):
    return f"Deck: {self.deck} \nDiscard: {self.discard} \nRunning Count: {self.rCount} \nPlayer Hand: {self.playerHand} \nDealer Hand: {self.dealerHand} \nRound Started: {self.roundStarted}"

  def __init__(self, numDecks):
    self.deck = [2,3,4,5,6,7,8,9,10,10,10,10,11] * 4 * numDecks #ingoring suits, not needed for blackjack
    self.rCount = 0 #Wont update for now, will just keep discrad pile and create method to update later
    self.discard = [] #Also contains cards that are in play (on the table)
    self.playerHand = []
    self.dealerHand = []
    
    self.shuffle()

  
  def shuffle(self):
    for card in self.discard:
      self.deck.append(card)
    self.discard = []
    #shuffle 5 times
    for i in range (5):
      random.shuffle(self.deck)

  def hand_value(self,hand):
    value = sum(hand)
    if value > 21 and 11 in hand:
      hand.remove(11)
      hand.append(1)
      value = sum(hand)
    return value
  

  #might need to change this account for aces that are already overdrawn for value 11
  def has_ace(self):
    return 11 in self.playerHand
  
  def deal_card(self):
    card = self.deck.pop()
    if card < 7:
      self.rCount += 1
    elif card > 9:
      self.rCount -= 1
    self.discard.append(card)
    return card
  
  def get_state(self):
    """
     Returns: player hand-val, dealers first card, hasAce, roundstarted(bbool), reward
     """
    pl = self.hand_value(self.playerHand)
    dl = self.dealerHand[0]
    runningCount = self.rCount
    reward = self.get_reward()
    roundStarted = self.roundStarted
    #Returns: player hand-val, dealers first card, hasAce, roundstarted(bbool), reward
    return (pl,dl,self.has_ace(), roundStarted,reward,runningCount)
  
  def get_reward(self):
    if self.roundStarted:
      return 0
    else:

      playerValue = self.hand_value(self.playerHand)
      dealerValue = self.hand_value(self.dealerHand)
      if playerValue > 21:
        return -1
      if dealerValue > 21:
        return 1
      if playerValue > dealerValue:
        return 1
      if playerValue < dealerValue:
        return -1
      return 0
    
  def end_round(self):
    
    if self.get_reward() == 0:
      print("Tie")
    elif self.get_reward() == 1:
      print("Player wins")
    else:
      print("Dealer wins")
    
    self.playerHand = []
    self.dealerHand = []

  def start_round(self):
    if len(self.deck) < 10:
      self.shuffle()
    
    if self.roundStarted:
      print("Round already started")
      return None
    self.roundStarted = True
    self.playerHand = [self.deal_card(), self.deal_card()]
    self.dealerHand = [self.deal_card(), self.deal_card()]
    return self.get_state()

  def next(self,action):
    if not self.roundStarted:
      print("Round not started")
      return None
    
    if action == 1:
      self.playerHand.append(self.deal_card())
      if self.hand_value(self.playerHand) > 21:
        self.roundStarted = False
        state = self.get_state()
        self.end_round()
        return state
      return self.get_state()
    else:
      
      while self.hand_value(self.dealerHand) < 17:
        self.dealerHand.append(self.deal_card())
      self.roundStarted = False
      state = self.get_state()
      self.end_round()
      return state
    #Add option to double also
  
     



  
  
  
    











In [25]:
table = BlackJackEnv(1)
print(table)


Deck: [7, 9, 5, 10, 6, 10, 8, 4, 6, 3, 10, 10, 10, 8, 2, 6, 4, 10, 9, 7, 3, 4, 3, 10, 11, 5, 10, 10, 10, 10, 9, 11, 7, 7, 10, 8, 5, 11, 2, 5, 10, 4, 10, 6, 11, 3, 9, 2, 10, 10, 8, 2] 
Discard: [] 
Running Count: 0 
Player Hand: [] 
Dealer Hand: [] 
Round Started: False


In [10]:

table.start_round()


(13, 10, False, True, 0, 1)

In [11]:
table.next(0)

Dealer wins


(13, 10, False, False, -1, 2)

In [9]:
print(table)

Deck: [10, 7, 3, 2, 10, 7, 9, 5, 4, 2, 10, 10, 10, 4, 5, 9, 4, 10, 2, 10, 10, 4, 10, 8, 8, 9, 10, 10, 6, 6, 10, 3, 7, 9, 3, 10, 10, 11, 8, 10, 5, 5, 10, 6, 7] 
Discard: [6, 11, 8, 2, 11, 3, 11] 
Running Count: 0 
Player Hand: [] 
Dealer Hand: [] 
Round Started: False


In [ ]:
 

def q_solve(env, iterations):
    

    #ROWS REPRESENT PLAYER TOTALS: 4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21 (18 total)
    #COLUMNS REPRESENT DEALER SHOWING: 2,3,4,5,6,7,8,9,10,11 (10 total)
    #HAS ACE: 0,1 (2 total)
    # ACTION: HIT, STAY (2 total)
    r = 18
    c = 10
    
    qVals = []
    #Initializing Qvals arr 
    for i in range(r):
        q_row = []
        for j in range(c):
            q_row.append([[0.0,0.0],[0.0,0.0]])
        
        qVals.append(q_row)
        
    #Learning
    for _ in range(iterations):
        #Random drop on the map
        r_drop = random.randint(0,r-1)
        c_drop = random.randint(0,c-1)
        mvs = 0
        pl,dl,hasAce, roundStarted,reward,runningCount = env.start_round()
        

        while True:
            #If we get dropped on p or g, its a terminal state and we end trial.
            if reward== 1:
                qVals[pl-4][dl-2] = [1.0, 1.0]
                break

            if reward== -1:
                qVals[pl-4][dl-2] = [-1.0, -1.0]
                break

            pr = random.random()
            possible_moves = [0,1] #add double later
            action = 0
            #WIth prob EXPLORE_PROB we pick a random possible move, otherwise we pick best q
            # Might also want to add always hit under 12 rule
            if pr < 0.8:
                action = random.choice(possible_moves)
            else:
                
                best_action_q = max(qVals[pl-4][dl-2])
                action = qVals[pl-4][dl-2].index(best_action_q)

            pl_new,dl_new,hasAce, roundStarted,reward,runningCount = env.next()
            #Calling role step function using the move we chose to get new coordinates
            #new_row,new_col = roll_steps(problem.move_probs,r_drop,c_drop,MOVES[action],r,c)
            #calling new_q to update current q
            q = new_q(reward,qVals,pl,dl,pl_new,dl_new,action)
            #Updating q in qvals arr
            qVals[pl-4][dl-2][action] = q
            #Moving on if not terminal state
            pl = pl_new
            dl= dl_new
            


    policy = Policy(problem)

    for i in range (r):
        for j in range(c):
            if problem.map[i][j]=="G" or  problem.map[i][j]=="P":
                continue
            best_action_index = qVals[i][j].index(max(qVals[i][j]))
            best_action = MOVES[best_action_index]
            policy.best_actions[i][j] = best_action


    return policy

#Helper function to return the possible moves from a specific square (to avoid going off map)
def get_pos_moves(problem,r,c):
    moves = []
    if r > 0:
        moves.append(0)
    if r < len(problem.map) - 1:
        moves.append(2)
    if c > 0:
        moves.append(3)
    if c < len(problem.map[0]) - 1:
        moves.append(1)
    return moves

#Helper function to get the reward for a coordinate
def get_reward(problem, r, c):
    if problem.map[r][c] == "G":
        return GOLD_REWARD
    elif problem.map[r][c] == "P":
        return PIT_REWARD
    else:
        return 0.0

#Q learning function uses
def new_q(action_reward, qVals, pl_old, dl_old, pl_new, dl_new, movenum):
    """ Q-learning function.  Returns the new Q-value for space (r,c).
    It's recommended you code and test this before doing the overall Q-learning.

    Should use the LEARNING_RATE and DISCOUNT_FACTOR.

    Args:
        rewards (List[List[float]]):  Reward amounts built into the problem map (indexed [r][c])
        utilities (List[List[List[float]]]):  The Q-values for each action from each space.
                                              (Indexed as [row][col][move])
        r, c (int, int):  Row and column of our location before move
        new_r, new_c (int, int):  Row and column of our location after move
        movenum (int):  Integer index into the Q-values, corresponding to constants UP etc
    Returns:
        float - the new Q-value for the space we moved from
    """
    old_q = qVals[pl_old-4][dl_old-2][movenum]
    
    best_next_q = max(qVals[pl_new][dl_new])

    new_q = old_q + LEARNING_RATE * (action_reward + DISCOUNT_FACTOR * best_next_q - old_q)


    return new_q